( 10 pts) Include any previous data preparation steps, EDA and visualizations. Its ok to copy and paste your code. However, ensure that you update the code based on the previous feedback from the TAs.

In [1]:
!pip install fastparquet

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

tripdata_df = pd.read_parquet(path = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2020-02.parquet', #provide the URL to the data source
                      engine = 'fastparquet')

tripdata_df.head()


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2020-02-01 00:10:25,2020-02-01 00:14:34,N,1.0,74,41,1.0,0.76,4.5,0.5,0.5,0.00,0.0,NaN,0.3,5.80,2.0,1.0,0.0
1,2,2020-02-01 00:16:59,2020-02-01 00:21:35,N,1.0,74,74,1.0,0.72,5.0,0.5,0.5,0.00,0.0,NaN,0.3,6.30,1.0,1.0,0.0
2,2,2020-02-01 00:19:31,2020-02-01 00:25:29,N,1.0,223,7,1.0,0.89,6.0,0.5,0.5,1.82,0.0,NaN,0.3,9.12,1.0,1.0,0.0
3,2,2020-02-01 00:43:52,2020-02-01 00:48:58,N,1.0,145,145,1.0,1.12,6.0,0.5,0.5,0.00,0.0,NaN,0.3,7.30,2.0,1.0,0.0
4,2,2020-02-01 00:32:53,2020-02-01 00:35:19,N,1.0,166,166,1.0,0.65,4.0,0.5,0.5,1.06,0.0,NaN,0.3,6.36,1.0,1.0,0.0


In [2]:
tripdata_df.isnull().sum(axis=1).sort_values(ascending=False)

398631    7
344706    7
344699    7
344700    7
344701    7
         ..
132875    1
132874    1
132873    1
132872    1
199316    1
Length: 398632, dtype: int64

In [3]:
#performing EDA plus TA recommended changes to EDA

#dropping 'ehail_fee'
tripdata_df = tripdata_df.drop(columns=['ehail_fee'])

#dropping all NaN values from the data set
tripdata_df = tripdata_df.dropna()

tripdata_df.head(75)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2020-02-01 00:10:25,2020-02-01 00:14:34,N,1.0,74,41,1.0,0.76,4.5,0.5,0.5,0.00,0.0,0.3,5.80,2.0,1.0,0.0
1,2,2020-02-01 00:16:59,2020-02-01 00:21:35,N,1.0,74,74,1.0,0.72,5.0,0.5,0.5,0.00,0.0,0.3,6.30,1.0,1.0,0.0
2,2,2020-02-01 00:19:31,2020-02-01 00:25:29,N,1.0,223,7,1.0,0.89,6.0,0.5,0.5,1.82,0.0,0.3,9.12,1.0,1.0,0.0
3,2,2020-02-01 00:43:52,2020-02-01 00:48:58,N,1.0,145,145,1.0,1.12,6.0,0.5,0.5,0.00,0.0,0.3,7.30,2.0,1.0,0.0
4,2,2020-02-01 00:32:53,2020-02-01 00:35:19,N,1.0,166,166,1.0,0.65,4.0,0.5,0.5,1.06,0.0,0.3,6.36,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2,2020-02-01 00:48:44,2020-02-01 01:09:36,N,1.0,112,198,1.0,4.31,16.5,0.5,0.5,3.56,0.0,0.3,21.36,1.0,1.0,0.0
71,2,2020-02-01 00:16:53,2020-02-01 00:22:27,N,1.0,255,256,1.0,0.98,5.5,0.5,0.5,1.36,0.0,0.3,8.16,1.0,1.0,0.0
72,2,2020-02-01 00:25:33,2020-02-01 00:44:19,N,1.0,256,225,1.0,3.23,14.5,0.5,0.5,0.00,0.0,0.3,15.80,2.0,1.0,0.0
73,2,2020-02-01 00:56:55,2020-02-01 01:01:15,N,1.0,49,225,1.0,1.01,5.5,0.5,0.5,0.00,0.0,0.3,6.80,2.0,1.0,0.0


In [4]:
tripdata_df['store_and_fwd_flag'].value_counts(dropna=False)

N    316828
Y       910
Name: store_and_fwd_flag, dtype: int64

In [5]:
tripdata_df['RatecodeID'].value_counts(dropna=False)

1.0     309477
5.0       7246
2.0        620
4.0        263
3.0        130
6.0          1
99.0         1
Name: RatecodeID, dtype: int64

In [6]:
tripdata_df['passenger_count'].value_counts(dropna=False)

1.0    273447
2.0     23121
5.0      9178
3.0      4927
6.0      4809
4.0      1775
0.0       464
7.0         9
8.0         8
Name: passenger_count, dtype: int64

In [7]:
tripdata_df['payment_type'].value_counts(dropna=False)

1.0    176529
2.0    138849
3.0      1767
4.0       579
5.0        14
Name: payment_type, dtype: int64

In [8]:
tripdata_df['trip_type'].value_counts(dropna=False)

1.0    310466
2.0      7272
Name: trip_type, dtype: int64

In [9]:
tripdata_df['congestion_surcharge'].value_counts(dropna=False)

0.00    263466
2.75     54205
2.50        66
0.75         1
Name: congestion_surcharge, dtype: int64

In [10]:
tripdata_df['total_amount'].value_counts(dropna=False)

7.30     9804
7.80     9582
6.80     9495
8.30     9103
6.30     8520
         ... 
43.47       1
15.76       1
19.06       1
23.77       1
30.03       1
Name: total_amount, Length: 4382, dtype: int64

In [11]:
#dropping negative values from the 'total_amount' column
tripdata_df = tripdata_df[tripdata_df.total_amount > 0]

(tripdata_df['total_amount'] < 0).sum().sum()



0

(5 pts) Select the required features to build your model based on the insights from your EDA. Briefly explain the reason for the features that you selected. Ensure that you encode any categorical features.

In [12]:
tripdata_df = pd.get_dummies(tripdata_df, drop_first=True)

In [13]:
#input features
X = tripdata_df[['trip_distance', 'total_amount', 'payment_type', ]]

#target
y = tripdata_df['tip_amount']



I chose the features 'trip_distance', 'total_amount', and 'payment_type' for the following reasons: A longer trip perhaps gives passengers more time to become friendly with a cab driver, thus influencing their tip. At the very least, a passenger will leave a tip commensurate to the amount that a driver has driven. The total amount of the cab fare will likely also play a role in how much a passenger leaves as a tip. Like in a restaurant, perhaps passengers feel as though they should leave some percentage of the total fare as a tip. Finally, I believe that payment type also plays  a role in how much of a tip is left. If a passenger pays with a card, perhaps they have little to no cash, and thus leave a small tip or none at all.

(5 pts) Partition the data into train/test split.

In [19]:

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size=0.3,
                                                   random_state=7,
                                                   )



(15 pts) Build a model that predicts the tip amount.

In [23]:
rf = RandomForestRegressor(random_state=7)
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=7)

In [24]:
y_pred   = rf.predict(X_test)

print('The predicted tip amount is: {}'.format(y_pred))

The predicted tip amount is: [0.         1.82643333 1.34817172 ... 1.46       1.9977     1.96      ]


(5 pts) Evaluate the predictions from your model and comment on the results. Ensure that you choose the correct metric. Remember that we evaluate models differently depending on the task, i.e. classification or regression.

In [25]:
mse = mean_squared_error(y_test, y_pred)

print('The mse of the model is: {}'.format(mse))

The mse of the model is: 4.112051191999708


This mean squared error doesn't indicate that the chosen features are great indicators of tip amount

(10 pts) How do you feel about the model? Does it do a good job of predicting the tip_amount?
Tweak the model: you can control the number of trees in your forest using the n_estimators parameter. Refer to the sample code from the lecture and also view the documentation. to learn more about this algorithm and its parameters. Try at least three different values for the n_estimators parameter to determine if the predictions improved (or did they get worse)? Show your steps.

I think that the model isn't accurate enough to be taken seriously. I'll tweak it by trying different n_estimators values to try to improve the predictions.

In [26]:
#n_estimators = 200
rf = RandomForestRegressor(random_state=7, n_estimators=200)
rf.fit(X_train, y_train)

y_pred   = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print('The mse of the model is: {}'.format(mse))

The mse of the model is: 4.006536404162277


In [27]:
#n_estimators = 300
rf = RandomForestRegressor(random_state=7, n_estimators=300)
rf.fit(X_train, y_train)

y_pred   = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print('The mse of the model is: {}'.format(mse))

The mse of the model is: 4.059405156417716


In [28]:
#n_estimators = 250
rf = RandomForestRegressor(random_state=7, n_estimators=250)
rf.fit(X_train, y_train)

y_pred   = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print('The mse of the model is: {}'.format(mse))

The mse of the model is: 4.008524614425164


In [30]:
#n_estimators = 225
rf = RandomForestRegressor(random_state=7, n_estimators=225)
rf.fit(X_train, y_train)

y_pred   = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print('The mse of the model is: {}'.format(mse))

The mse of the model is: 3.993748172988104


In [33]:
#n_estimators = 217
rf = RandomForestRegressor(random_state=7, n_estimators=217)
rf.fit(X_train, y_train)

y_pred   = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print('The mse of the model is: {}'.format(mse))

The mse of the model is: 4.000811063392419


In [34]:
#n_estimators = 237
rf = RandomForestRegressor(random_state=7, n_estimators=237)
rf.fit(X_train, y_train)

y_pred   = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print('The mse of the model is: {}'.format(mse))

The mse of the model is: 4.011620993220357


It looks like changing the n_estimators to 225 gives the best mse result. Still though, not a convincing model.

(+5 bonus points) The Random forest has a method that returns the importance of each feature in your model. Can you find out which of your selected features were the most important when making the predictions?

In [31]:
rf.feature_importances_

array([0.24243185, 0.48878329, 0.26878486])

It appears that total_amount is the strongest predictor of tip_amount. This was expected, however I'm surprised at how little the significance is. 